In [53]:
import pandas as pd
import ast
import re

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules



In [ ]:
df = pd.read_csv('cleaned_jobs.csv')


In [17]:
df.head(4).T

,0,1,2,3
job_id,16d869d5-0bb2-4c6d-bac3-25d00a94c7d8,c12e7ca2-cfae-4eeb-9091-09d5c6c2d655,9d52e7a2-8ac9-43b2-83b2-8c25e301589f,328a7231-1192-4ea4-953f-b07377d7f5b4
date,2025-02-10,2024-05-16,2024-04-01,2025-04-03
job_title,DevOps Engineer,Data Analyst,Data Analyst,Data Scientist
experience_level,Junior,Expert,Midlevel,Senior
city,Tehran,Isfahan,Kashan,Kerman
remote_option,False,False,False,False
company_size,Medium,Small,Small,Small
gender,Male,Male,Male,Male
skills_required,"['CI/CD', 'Linux Administration', 'Monitoring ...","['Excel', 'Presentation', 'Data Analysis']","['Excel', 'Presentation', 'PowerBI']","['AI', 'Python']"
salary_million_tmn,22.49,64.01,30.98,41.97


In [30]:
df.skills_required[6981]

"['Mathematics (Linear Algebra, Calculus)', 'Big Data', 'AI', 'Hadoop']"

In [ ]:
# Function to parse and separate the list string
def parse_skills(skill_string):
    # Remove outer brackets and extract quoted strings
    cleaned = skill_string.strip("[]")
    elements = re.findall(r"'([^']*)'", cleaned)
    result = []
    for element in elements:
        if '(' in element:
            # Split into main term and parenthetical content
            main, rest = element.split(' (')
            rest = rest.rstrip(')')
            # Add main term and split parenthetical content
            result.append(main.strip())
            result.extend(x.strip() for x in rest.split(','))
        else:
            result.append(element.strip())
    return result

df['skills_required'] = df['skills_required'].apply(parse_skills)

In [45]:
df.iloc[1148].skills_required

['Version Control',
 'Git',
 'Database Management',
 'SQL/NoSQL',
 'Responsive Design',
 'Frontend Frameworks',
 'React',
 'Angular',
 'Vue.js',
 'Backend Development',
 'Node artístico.js',
 'Django',
 'Ruby on Rails']

In [52]:

# Check if TransactionEncoder is correctly initialized
te = TransactionEncoder()
te_ary = te.fit(df['skills_required']).transform(df['skills_required'])
df_te = pd.DataFrame(te_ary, columns=te.columns_)

df_te.head(3).T


,0,1,2
AI,False,False,False
API Development,False,False,False
AWS,True,False,False
Agile,False,False,False
Android,False,False,False
...,...,...,...
VPN Setup,False,False,False
Version Control,False,False,False
Vue.js,False,False,False
e.g.,False,False,False


In [54]:
frequent_itemsets = apriori(df_te, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)


In [58]:
rules.head(3).T

,0,1,2
antecedents,(Big Data),(AI),(AI)
consequents,(AI),(Big Data),(Calculus)
antecedent support,0.131389,0.099722,0.099722
consequent support,0.099722,0.131389,0.048889
support,0.046111,0.046111,0.020278
confidence,0.350951,0.462396,0.203343
lift,3.51929,3.51929,4.159281
representativity,1.0,1.0,1.0
leverage,0.033009,0.033009,0.015402
conviction,1.387073,1.615707,1.193877
